In [18]:
#pip install tensorflow

Nesse projeto uso LSTM para descobrir qual é o preço de fechamento do bitcoin. Uso 5 features relativas ao candle de negociação diária e prevejo o preço de fechamento do dia seguinte



In [19]:
# Importar as bibliotecas necessárias
import os
import pandas as pd
import numpy as np
import math
import datetime
import time

# Importar sklearn para avaliar o modelo
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing  import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix


# Importar tensorflow para construir o modelo
import tensorflow as tf
import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, History, EarlyStopping
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

# Importar bibliotecas para plotar gráficos

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



## Variaveis Globais

In [20]:
TIME_STEP = 15
JUMP_STEP = 1
NUM_FEATURES = 5
# usando apenas o Close


## Importar a Base de Dados

In [21]:
ticker = 'BTC-USD'
#1410825600
initial_period = int(time.mktime(datetime.datetime(2014, 9, 16, 0, 0).timetuple()))

#1675814400
final_period = int(time.mktime(datetime.datetime(2023, 2, 8, 0, 0).timetuple()))
# 3067 valores. 2300 é 75%
interval = '1d' # 1d, 1wk, 1m

query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={initial_period}&period2={final_period}&interval={interval}&events=history&includeAdjustedClose=true'

data = pd.read_csv(query_string)
df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.index = pd.to_datetime(df['Date'], format='%Y-%m-%d')


Função para escolher os dados utilizados de forma sequencial. A cada t valores


In [22]:
def selecionaValores(df, jump_step):
  return df[0::jump_step]

df = selecionaValores(df, JUMP_STEP)

Preparação dos Dados:

In [23]:
# Retira a Data e coloca como Index
df.index = df.pop('Date')

# Adição da coluna de variação
df['TargetChange'] = df.Close.shift(-1) - df['Close']
# Adição da coluna de variação percentual
df['TargetVariation'] = (df['TargetChange'] / df['Close'])*100
# Adição da coluna de classificação
# Adição da coluna de Valor Final
df['TargetClose'] = df['Close'].shift(-1)

# df.pop('TargetVariation')
# df.pop('TargetClass')
# df.pop('TargetClose')
data.dropna(inplace=True)
data.reset_index(inplace = True)


In [24]:
df.head()

,Open,High,Low,Close,Volume,TargetChange,TargetVariation,TargetClose
Date,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,-32.894013,-7.192558,424.440002
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,-29.644012,-6.984264,394.795990
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,14.108002,3.573492,408.903992
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,-10.082978,-2.465855,398.821014
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,3.330994,0.835210,402.152008


In [25]:
#plt.plot(df.index, df['Close'])

In [26]:
# Separa as Features do Valor Final
# Separa X de Y e Retira a última linha
#df.pop('Open')
#df.pop('High')
#df.pop('Low')
#df.pop('Volume')

#y_array = np.array(df.TargetVariation)[:-1].copy()
df.pop('TargetChange')
#df.pop('TargetVariation')
df.pop('TargetClose')
x_array = np.array(df)[:-1].copy()


In [27]:
# Divisão entre 70% treino, 10% validação e 20% teste
size = len(df-1) - TIME_STEP
train_size = int(size * 0.7)
validation_size = int(size * 0.1)
test_size = size - train_size - validation_size

train_data = x_array[: train_size]
val_data = x_array[train_size : (train_size + validation_size)]
test_data = x_array[(train_size + validation_size) :]


In [28]:
#Transforma os dados para suavizar as variações em escala absoluta
# Transform input data
scaler_train_input = MinMaxScaler(feature_range=(0, 1))
scaled_train_input = scaler_train_input.fit_transform(train_data[:,:-1])

scaler_val_input = MinMaxScaler(feature_range=(0, 1))
scaled_val_input = scaler_val_input.fit_transform(val_data[:,:-1])

scaler_test_input = MinMaxScaler(feature_range=(0, 1))
scaled_test_input = scaler_test_input.fit_transform(test_data[:,:-1])

# Transform output data
scaler_train_output = MinMaxScaler(feature_range=(0, 1))
scaled_train_output = scaler_train_output.fit_transform(train_data[:,-1].reshape(-1, 1))

scaler_val_output = MinMaxScaler(feature_range=(0, 1))
scaled_val_output = scaler_val_output.fit_transform(val_data[:,-1].reshape(-1, 1))

scaler_test_output = MinMaxScaler(feature_range=(0, 1))
scaled_test_output = scaler_test_output.fit_transform(test_data[:,-1].reshape(-1, 1))

In [29]:
def createDataset(x, y, time_step=1):
  input = []
  output = []
  for i in range(len(x) - time_step):
    row = [e for e in x[i : i + time_step]]
    input.append(row)
    output.append(y[i])
  return np.array(input), np.array(output)

x_train, y_train = createDataset(scaled_train_input, scaled_train_output, TIME_STEP)
x_val, y_val = createDataset(scaled_val_input, scaled_val_output, TIME_STEP)
x_test, y_test = createDataset(scaled_test_input, scaled_test_output, TIME_STEP)

print(df.shape)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(3067, 6)
(2121, 15, 5) (2121, 1)
(290, 15, 5) (290, 1)
(610, 15, 5) (610, 1)


## Preparar os dados para Treino e Teste



## Construção do Modelo LSTM

### Modelo: LSTM



In [30]:
#create the sequence model
model = Sequential()

model.add(LSTM(units = 128, activation = 'relu',return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 64, input_shape = (x_train.shape[1], x_train.shape[2])))
#model.add(LSTM(units = 64, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

# model.add(LSTM(units = 64, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
# model.add(Dropout(0.2))

# model.add(LSTM(units = 64, input_shape = (x_train.shape[1], x_train.shape[2])))
# model.add(Dropout(0.2))

model.add(Dense(units=1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 15, 128)           68608     
                                                                 
 dropout_2 (Dropout)         (None, 15, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 118,081
Trainable params: 118,081
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
checkpoint_path = 'checkpoint_tccModelo2.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
earlystopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [checkpoint, earlystopping]
# callbacks = [checkpoint]

history = model.fit(x_train, y_train, batch_size = 32, epochs = 100, verbose=1, shuffle=False, validation_data=(x_val, y_val), callbacks=callbacks)




Epoch 1/100
65/67 [============================>.] - ETA: 0s - loss: 0.0062
Epoch 1: val_loss improved from inf to 0.03478, saving model to checkpoint_tccModelo2.hdf5
67/67 [==============================] - 7s 49ms/step - loss: 0.0062 - val_loss: 0.0348
Epoch 2/100
67/67 [==============================] - ETA: 0s - loss: 0.0056
Epoch 2: val_loss did not improve from 0.03478
67/67 [==============================] - 2s 32ms/step - loss: 0.0056 - val_loss: 0.0432
Epoch 3/100
66/67 [============================>.] - ETA: 0s - loss: 0.0065
Epoch 3: val_loss did not improve from 0.03478
67/67 [==============================] - 4s 55ms/step - loss: 0.0065 - val_loss: 0.0539
Epoch 4/100
67/67 [==============================] - ETA: 0s - loss: 0.0059
Epoch 4: val_loss did not improve from 0.03478
67/67 [==============================] - 2s 32ms/step - loss: 0.0059 - val_loss: 0.0382
Epoch 5/100
67/67 [==============================] - ETA: 0s - loss: 0.0054
Epoch 5: val_loss did not improve fr

## Previsões

In [ ]:
from tensorflow.keras.models import load_model

model_from_saved_checkpoint = load_model(checkpoint_path)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
test_predict = model_from_saved_checkpoint.predict(x_test)

test_predict = scaler_test_output.inverse_transform(test_predict.reshape(-1, 1))

test_actual = scaler_test_output.inverse_transform(y_test.reshape(-1, 1))
plt.figure(figsize=(16,7))

plt.plot(test_predict, 'r', marker='.', label='Predicted Test', linewidth=0)

plt.plot(test_actual, marker='.', label='Actual Test',linewidth=0)

plt.legend()
plt.show()

Plotar o Gráfico